```
Check for duplicates (Primary Key) .
```

In [ ]:
SELECT
    location_id, COUNT(1) AS numbers_of_records
FROM
    OPENROWSET(
        BULK 'abfss://nyc-taxi-data@wsmaindl.dfs.core.windows.net/raw/taxi_zone.csv',
        FORMAT = 'CSV',
        PARSER_VERSION = '2.0',
        FIRSTROW = 2,
        FIELDTERMINATOR = ',',
        ROWTERMINATOR = '\n'
    ) 
    WITH (
        location_id SMALLINT 1,
        borough VARCHAR(15) 2,
        zone VARCHAR(50) 3,
        service_zone  VARCHAR(15) 4 
    )AS [result]
    GROUP BY location_id
    HAVING COUNT(1) > 1;

Data quality checks for trip totoal amount

In [ ]:
SELECT
    MAX(total_amount) AS max_total_amount,
    MIN(total_amount) AS min_total_amount,
    AVG(total_amount) AS avg_total_amount,
    COUNT(1) AS total_records,
    COUNT(total_amount) AS not_null_total_number_of_records
FROM
    OPENROWSET(
        BULK 'trip_data_green_parquet/year=2020/month=01/',
        FORMAT = 'PARQUET',
        DATA_SOURCE = 'nyc_taxi_data_raw'    

    ) AS [result]

Check why totoal amount has negative values.

In [8]:
SELECT
    payment_type, COUNT(1)
    
FROM
    OPENROWSET(
        BULK 'trip_data_green_parquet/year=2020/month=01/',
        FORMAT = 'PARQUET',
        DATA_SOURCE = 'nyc_taxi_data_raw'    

    ) AS [result]
    --WHERE total_amount < 0
    GROUP BY payment_type
    ORDER BY payment_type;

Statement ID: {06383632-5290-4642-95EB-8FC5BAF7E80A} | Query hash: 0xC962DA78CD50125C | Distributed request ID: {886D934B-252A-4DC6-97CB-49A15A683A0A}. Total size of data scanned is 1 megabytes, total size of data moved is 1 megabytes, total size of data written is 0 megabytes.

(6 rows affected)

Total execution time: 00:00:00.293

payment_type,(No column name)
NULL,116051
1,184404
2,145027
3,1691
4,581
5,16


Check payment type

In [9]:
SELECT payment_type, description
FROM
    OPENROWSET(
        BULK 'payment_type.json',
        DATA_SOURCE = 'nyc_taxi_data_raw',
        FORMAT = 'CSV',
        FIELDTERMINATOR = '0x0b',
        FIELDQUOTE = '0x0b'
    ) WITH(
        jsonDoc NVARCHAR(MAX)
    )AS payment_type
    CROSS APPLY OPENJSON(jsonDoc) --INNER JOIN WITHOUT ANY JOIN CONDITION.
    WITH(
        payment_type SMALLINT,
        description VARCHAR(20) '$.payment_type_desc'
    );

Statement ID: {4E719245-3E57-4401-AA8A-E0F1A8C57273} | Query hash: 0xE9E175CCEF225A4E | Distributed request ID: {DEE8BAE3-E153-4817-9750-B23C5E403E1C}. Total size of data scanned is 1 megabytes, total size of data moved is 1 megabytes, total size of data written is 0 megabytes.

(6 rows affected)

Total execution time: 00:00:00.385

payment_type,description
1,Credit card
2,Cash
3,No charge
4,Dispute
5,Unknown
6,Voided trip
